In [ ]:
from PIL import Image
from skimage import measure                       
from shapely.geometry import Polygon, MultiPolygon 
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from Utils.training import iou_np
from Utils.build_dataset import paths_dataset_shrimp
from Utils.coco_dataset import get_annotations, create_sub_mask_annotation
from torchvision import transforms
from Utils.training import training_loop, predict, iou_np
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.utils.visualizer import Visualizer
from detectron2.config import get_cfg
from detectron2 import model_zoo
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd
import time
import cv2
import os

## Construindo o Dataset

In [ ]:
path = 'Dataset/Fish_Dataset/Fish_Dataset/Shrimp/'
df_train_pd, df_valid_pd, df_test_pd = paths_dataset_shrimp(path)

In [ ]:
DatasetCatalog.register('shrimp_train', lambda: get_annotations(df_train_pd))
MetadataCatalog.get('shrimp_train').set(thing_classes=['shrimp'])
#fish_metadata = MetadataCatalog.get("fish_train")

In [ ]:
DatasetCatalog.register('shrimp_valid', lambda: get_annotations(df_valid_pd))
MetadataCatalog.get('shrimp_valid').set(thing_classes=['shrimp'])

In [ ]:
DatasetCatalog.register('shrimp_test', lambda: get_annotations(df_test_pd))
MetadataCatalog.get('shrimp_test').set(thing_classes=['shrimp'])

## Treinando o Modelo

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("shrimp_train",)
cfg.DATASETS.TEST = ("shrimp_valid",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.0001  # pick a good LR
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)

In [ ]:
exp_number = 1 #id do experimento, será usado para salvar o modelo com nome único
n_exps = 30 #Quantidade de experimentos que serão executados
path_metrics = 'Metrics' #Path da pasta onde será salvo o arquivo csv com as métricas
iou_dict = {}

In [ ]:
training_start_time = time.time()
for n in range(exp_number, (n_exps + 1)):
    cfg.OUTPUT_DIR = f'output_detectron2/exp_{n}'
    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    trainer = DefaultTrainer(cfg) 
    trainer.resume_or_load(resume=False)
    trainer.train()
    
    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4
    predictor = DefaultPredictor(cfg)
    
    df_test = get_annotations(df_test_pd)
    
    iou_sum = 0
    iou_list = []
    for annon in df_test:
        im = cv2.imread(annon['file_name'])
        outputs = predictor(im)
        try:
            pred = outputs['instances'].pred_masks[0]
            pred = pred.to('cpu')
            pred_int = pred.long()
            pred_np = pred_int.numpy()
            iou_pred = iou_np(annon['mask'], pred_np)
        except:
            iou_pred = 0
        iou_sum += iou_pred
        iou_list.append(iou_pred)

    iou = (iou_sum / len(df_test)) * 100
    iou_dict[n] = iou

print('Training finished, took {:.2f}s'.format(time.time() - training_start_time))
duration = time.time() - training_start_time

In [ ]:
duration / (30*60)

In [ ]:
iou_list = []
for value in iou_dict.values():
    iou_list.append(value)

In [ ]:
metrics_dict = {'iou': iou_list}
df_metrics = pd.DataFrame(metrics_dict)
df_metrics.shape
df_metrics.head()

In [ ]:
df_metrics['iou'].mean()

In [ ]:
path_csv = f'{path_metrics}/metrics_detectron2_segmentation_shrimp.csv'
df_metrics.to_csv(path_csv, sep='\t')